# <center>Discriminative Regularize Generative Model for CIFAR10 </center>

## Load Data

In [1]:
!pip3 install --upgrade torch torchvision

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython.display import Image
#from google.colab import files

#Set random seed 
torch.manual_seed(512)

    100% |████████████████████████████████| 484.0MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5cd40000 @  0x7f0e456901c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 15.7MB/s 
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.11.0)
    100% |████████████████████████████████| 2.0MB 16.4MB/s 
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.14.3)
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
class preTrainedModel(nn.Module):
  
    def __init__(self):
      
      super(preTrainedModel,self).__init__()
      
      vgg_model = torchvision.models.vgg16(pretrained=True)		
      self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
      #self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9]) 
      #self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
      #self.upSample1 = nn.Upsample(scale_factor=2)
      #self.upSample2 = nn.Upsample(scale_factor=4)

    def forward(self,x):
      out1 = self.Conv1(x)
      #out2 = self.Conv2(out1)
      #out3 = self.Conv3(out2)
      ###### up sampling to create output with the same size
      #out2 = self.upSample1(out2)
      #out3 = self.upSample2(out3)
      #concat_features = torch.cat([out1, out2, out3], 1)
      return out1

In [3]:
#Load model 
vgg19 = preTrainedModel().eval().cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /content/.torch/models/vgg16-397923af.pth
1.2%

100.0%


In [4]:
#Get the CIFAR10 train images 
cifar = datasets.CIFAR10('./data/cifar/', train = True, download = True)

# Organize training data in batches, 
# normalize them to have values between [-1, 1] (?)

train_images = torch.utils.data.DataLoader ( datasets.CIFAR10('./data/cifar/', train = True, download=False,
                               transform=transforms.Compose([
                               #transforms.Resize(64), 
                               #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               transforms.ToTensor(),])) , 
                               batch_size = 8, shuffle = True)

In [5]:
upsampling = nn.Upsample(size=256)
for batch_idx, (data,_) in enumerate(train_images):    
    out = vgg19(upsampling(data.cuda()))
    print(out.size())
    print(data.size())
    #print(data[0])
    break

torch.Size([8, 64, 256, 256])
torch.Size([8, 3, 32, 32])


## Model

We will use the arquitecture suggested by [Radford et al](https://arxiv.org/abs/1511.06434) for both the encoder and decoder. With convolutional layers in the encoder and fractionally-strided  convolutions  in  the  decoder.   In  each convolutional layer in the encoder we double the number of filters present in the previous layer and use a convolutional stride of 2.  In each convolutional layer in the decoder we use a fractional stride of 2 and halve the number of filters on each layer.

In [0]:
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

class VAE( nn.Module ):

    def __init__ ( self, image_size ,  hidden_dim , encoding_dim ):
        
        super( VAE, self ).__init__()
        
        self.encoding_dim = encoding_dim
        self.image_size = image_size
        self.hidden_dim = hidden_dim 
        
        # Decoder - Fractional strided convolutional layers
        self.decoder  = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 1, 0, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias = False),
            nn.Sigmoid() # nn.Tanh()  
        )
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(32, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 0, bias = False),
            nn.Sigmoid()
        )
        
        # Fully-connected layers
        self.fc1 = nn.Linear(256, self.hidden_dim)
        self.fc21 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc22 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc3 = nn.Linear(self.encoding_dim, self.hidden_dim)
        self.fc4 = nn.Linear(self.hidden_dim, 256)
    
    def decode (self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.sigmoid(self.fc4(h3))
        return self.decoder( h4.view(z.size(0),-1,1,1) ) 

        
    def forward(self, x):
        
        # Encode 
        encoded = F.relu(self.fc1( self.encoder(x).view(x.size(0), -1) ) )
        
        #Obtain mu and logvar
        mu = self.fc21( encoded )
        logvar = self.fc22 ( encoded )
        
        #Reparametrization trick
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(mu)
        
        # Decode 
        decoded = self.decode(z)

        # return decoded, mu, logvar
        return decoded, mu , logvar


upsampling = nn.Upsample(size=256)
sigmoid = nn.Sigmoid()

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    d1_recon_x = sigmoid(vgg19(upsampling( recon_x )))
    d1_x = sigmoid(vgg19( upsampling( x )))

    L1 = F.mse_loss(d1_recon_x, d1_x, size_average=False)
    
    del d1_recon_x 
    del d1_x
    
    return BCE + KLD + L1

In [8]:
from google.colab import files
uploaded = files.upload()

#Define model
model = VAE( 32, 100, 20 ).cuda()
#model.load_state_dict(torch.load('save_checkpoint_epoch_69.pth'))
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#Train model
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_images):
        data = Variable(data).cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_images.dataset),
                100. * batch_idx / len(train_images),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_images.dataset)))

Saving save_checkpoint_epoch_69.pth to save_checkpoint_epoch_69.pth


In [9]:
num_epochs = 2
for epoch in range(1,num_epochs):
    train(epoch)    

Train Epoch: 1 [0/50000 (0%)]	Loss: 7168.814453
Train Epoch: 1 [400/50000 (1%)]	Loss: 8286.398438
Train Epoch: 1 [800/50000 (2%)]	Loss: 8785.705078
Train Epoch: 1 [1200/50000 (2%)]	Loss: 8239.338867
Train Epoch: 1 [1600/50000 (3%)]	Loss: 8859.353516
Train Epoch: 1 [2000/50000 (4%)]	Loss: 7408.976562
Train Epoch: 1 [2400/50000 (5%)]	Loss: 6711.461914
Train Epoch: 1 [2800/50000 (6%)]	Loss: 6423.336426
Train Epoch: 1 [3200/50000 (6%)]	Loss: 8909.876953
Train Epoch: 1 [3600/50000 (7%)]	Loss: 6957.336914
Train Epoch: 1 [4000/50000 (8%)]	Loss: 8573.662109
Train Epoch: 1 [4400/50000 (9%)]	Loss: 6625.413086
Train Epoch: 1 [4800/50000 (10%)]	Loss: 8149.950195
Train Epoch: 1 [5200/50000 (10%)]	Loss: 8172.897461
Train Epoch: 1 [5600/50000 (11%)]	Loss: 7469.384766
Train Epoch: 1 [6000/50000 (12%)]	Loss: 7907.474121
Train Epoch: 1 [6400/50000 (13%)]	Loss: 7876.250977
Train Epoch: 1 [6800/50000 (14%)]	Loss: 6129.447266
Train Epoch: 1 [7200/50000 (14%)]	Loss: 8253.538086
Train Epoch: 1 [7600/50000 (1

Train Epoch: 1 [24400/50000 (49%)]	Loss: 7598.856445
Train Epoch: 1 [24800/50000 (50%)]	Loss: 7275.869141
Train Epoch: 1 [25200/50000 (50%)]	Loss: 6732.260254
Train Epoch: 1 [25600/50000 (51%)]	Loss: 8605.176758
Train Epoch: 1 [26000/50000 (52%)]	Loss: 7506.098633
Train Epoch: 1 [26400/50000 (53%)]	Loss: 6038.569824
Train Epoch: 1 [26800/50000 (54%)]	Loss: 6971.148438
Train Epoch: 1 [27200/50000 (54%)]	Loss: 7233.091797
Train Epoch: 1 [27600/50000 (55%)]	Loss: 6783.173340
Train Epoch: 1 [28000/50000 (56%)]	Loss: 7911.638672
Train Epoch: 1 [28400/50000 (57%)]	Loss: 8042.156250
Train Epoch: 1 [28800/50000 (58%)]	Loss: 7496.097656
Train Epoch: 1 [29200/50000 (58%)]	Loss: 7708.732910
Train Epoch: 1 [29600/50000 (59%)]	Loss: 7672.047363
Train Epoch: 1 [30000/50000 (60%)]	Loss: 7319.247070
Train Epoch: 1 [30400/50000 (61%)]	Loss: 6921.179199
Train Epoch: 1 [30800/50000 (62%)]	Loss: 8524.701172
Train Epoch: 1 [31200/50000 (62%)]	Loss: 7772.747559
Train Epoch: 1 [31600/50000 (63%)]	Loss: 6067.

Train Epoch: 1 [48400/50000 (97%)]	Loss: 7949.773438
Train Epoch: 1 [48800/50000 (98%)]	Loss: 7258.249512
Train Epoch: 1 [49200/50000 (98%)]	Loss: 8273.482422
Train Epoch: 1 [49600/50000 (99%)]	Loss: 8559.158203
====> Epoch: 1 Average loss: 7561.2935


In [0]:

torch.save(model.cpu().state_dict(), "save_checkpoint_epoch_70.pth")
files.download("save_checkpoint_epoch_70.pth")
               
               

In [11]:
with torch.no_grad():
        sample = torch.randn(64, 20)
        sample = model.decode(sample)
        #torch.save(model.cpu().state_dict(), "./save_checkpoint_epoch_"+str(epoch)+".pth")
        #files.download("./save_checkpoint_epoch_"+str(epoch)+".pth")
        torchvision.utils.save_image(sample.view(64, 3, 32, 32),'./sample_' + str(epoch) + '.png')
        files.download('./sample_' + str(epoch) + '.png')

AttributeError: ignored

In [0]:
torch.save(sample.cpu(), "sample_70.pth")
files.download("sample_70.pth")

In [18]:
sample.size()

torch.Size([64, 3, 32, 32])